In [2]:
import pandas as pd
import matplotlib as plt

In [3]:
data = pd.read_csv('anomaly_data.csv')


In [6]:
for i in range(len(data)):
    if (data.loc[i, 'Pressure (psi)'] < 27 or data.loc[i, 'Pressure (psi)'] > 150 or
        data.loc[i, 'Temperature (°C)'] > 88 or
        data.loc[i, 'Oil Level (%)'] < 18 or
        data.loc[i, 'Tool RPM (rpm)'] > 2500 and
        data.loc[i,'anomaly'] == -1):
        data.loc[i, 'Machine Status'] = 'NotRunning'

In [7]:
data.head()

,Unnamed: 0,Timestamp,Pressure (psi),Temperature (°C),Oil Flow (L/min),Vibration (mm/s),Power Status,Oil Level (%),Tool RPM (rpm),Machine Status,Current (A),scores,anomaly
0,0,17-09-2023 00:00,45.2,29.8,2.4,0.021,ON,73.5,1505,Running,10.3,0.036549,1
1,1,17-09-2023 00:00,48.2,28.8,3.4,0.031,ON,70.5,1805,Running,11.3,0.085057,1
2,2,17-09-2023 00:01,45.1,29.9,2.5,0.022,ON,73.3,1508,Running,10.4,0.040864,1
3,3,17-09-2023 00:01,48.1,28.9,3.5,0.032,ON,70.3,1808,Running,11.4,0.092148,1
4,4,17-09-2023 00:02,45.3,30.0,2.6,0.023,ON,73.2,1512,Running,10.5,0.046241,1


In [8]:
data.to_csv('data_aug.csv', index=False) 

In [ ]:
data['Vibration (mm/s)'].median()

0.6

In [7]:
import streamlit as st
import numpy as np
import pandas as pd
import time
import joblib as jb
from sklearn.pipeline import Pipeline
import plotly.express as px

st.set_page_config(
    page_title="Condition Monitoring",
    page_icon='H'
)



data = pd.read_csv('demonstrate_data.csv')

health = jb.load('predict.pkl')
prep = jb.load('prep.pkl')
anomaly = jb.load('anomaly.pkl')

features = ['Pressure (psi)',	'Temperature (°C)',	'Oil Flow (L/min)',	'Vibration (mm/s)','Oil Level (%)'	,'Tool RPM (rpm)' ,'Current (A)']
health_ft = ['Temperature (°C)','Tool RPM (rpm)']
pressure_cond = 'Normal'
temp_cond = 'Normal'
oil_cond = 'Normal'
rpm_cond = 'Normal'


def checks(i):

    pressure_psi = data.loc[i,'Pressure (psi)']
    temperature_c = data.loc[i,'Temperature (°C)']
    vibration_mm_s = data.loc[i,'Vibration (mm/s)']
    oil_level_percent = data.loc[i,'Oil Level (%)']
    tool_rpm_rpm = data.loc[i,'Tool RPM (rpm)']

    global pressure_cond, temp_cond, oil_cond, rpm_cond,vib_cond
    



    if pressure_psi > data['Pressure (psi)'].quantile(0.98):
        pressure_cond = 'Warning! Too High'
    elif pressure_psi < data['Pressure (psi)'].quantile(0.02):
        pressure_cond = 'Warning! Too Low'
    else:
        pressure_cond = 'Normal'

    if temperature_c > data['Temperature (°C)'].quantile(0.98):
        temp_cond = 'Warning! Too High'
    else:
        temp_cond = 'Normal'

    if oil_level_percent < data['Oil Level (%)'].quantile(0.02):
        oil_cond = 'Warning! Too Low'
    else:
        oil_cond = 'Normal'

    if tool_rpm_rpm > data['Tool RPM (rpm)'].quantile(0.98):
        rpm_cond = 'Warning! Too High'
    elif tool_rpm_rpm < data['Tool RPM (rpm)'].quantile(0.02):
        rpm_cond = 'Warning! Too Low'
    else:
        rpm_cond = 'Normal'
    
    if vibration_mm_s > data['Vibration (mm/s)'].quantile(0.98):
        vib_cond = 'Warning! Too High'
    elif vibration_mm_s < data['Vibration (mm/s)'].quantile(0.02):
        vib_cond = 'Warning! Too Low'
    else:
        vib_cond = 'Normal'

    return pressure_cond,temp_cond,oil_cond,rpm_cond

def predict_health(i):
    k = pd.DataFrame(data.loc[i,health_ft]).T
    feature_map = {'Temperature (°C)':'Process temperature','Tool RPM (rpm)':'Rotational speed'}
    features = ['Process temperature','Rotational speed']
    k = k.rename(columns=feature_map)
    k = prep.fit_transform(k)
    pred = health.predict(k)
    if pred != 0:
        x = 'Likely'
    else:
        x = 'Unlikely'
    return x
    
    


def predict_anomaly(i):
    pred = anomaly.predict(pd.DataFrame(data.loc[i,features]).T)
    if pred==-1:
        x = 'Abnormal'
    else:
        x = 'Normal'
    return x




st.title("Condition Monitoring")
update_interval = 1
update_interval = st.slider(min_value=1,max_value=5,value=None,label='Speed')

placeholder = st.empty()

status = 'sdf'

run = pd.DataFrame(columns=data.columns)

for i in range(0,len(data)):
    checks(i)
    with placeholder.container():      
        element1,element2,element3 = st.columns(3)
        e4, e5, e6 = st.columns(3)
        e7, e8 = st.columns(2)
        e9, e10 = st.columns(2)
        d1,d2= st.columns(2)
        d3,d4= st.columns(2)
        d5= st.empty()
        machine_health = st.empty()
        fig_col1, fig_col2 = st.columns(2)
        element1.metric(value=data.loc[i,'Pressure (psi)'],label='Pressure (psi)',delta=(data.loc[i,'Pressure (psi)']-data.loc[i-1,'Pressure (psi)']))
        element2.metric(value=data.loc[i,'Temperature (°C)'],label='Temperature (°C)')
        element3.metric(value=data.loc[i,'Oil Flow (L/min)'],label='Oil Flow (L/min)')
        e4.metric(value=data.loc[i,'Vibration (mm/s)'],label='Vibration (mm/s)')
        e5.metric(value=data.loc[i,'Power Status'],label='Power Status')
        e6.metric(value=data.loc[i,'Oil Level (%)'],label='Oil Level (%)')
        e7.metric(value=data.loc[i,'Tool RPM (rpm)'],label='Tool RPM (rpm)')
        e8.metric(value=data.loc[i,'Current (A)'],label='Current (A)')
        e9.metric(value=data.loc[i,'Machine Status'],label='Machine Status')
        e10.metric(value=predict_anomaly(i),label='Anomaly')
        d1.write(f"Pressure Levels: {pressure_cond}")
        d2.write(f"Temperature Levels: {temp_cond}")
        d3.write(f"Oil Levels: {oil_cond}")
        d4.write(f"RPM Levels: {rpm_cond}")
        d5.write(f"Vibration Levels: {vib_cond}")
        machine_health.metric(value=predict_health(i),label='Temperature/RPM Failure')

        
        with fig_col1:
            st.markdown("### First Chart")
            fig = px.bar(data_frame=data, y = 'Pressure (psi)')
            st.write(fig)
        with fig_col2:
            st.markdown("### Second Chart")
            fig2 = px.histogram(data_frame = data, y = 'Temperature (°C)')

        
        time.sleep(1/update_interval)

KeyboardInterrupt: 

In [13]:
data = pd.read_csv('demonstrate_data.csv')

In [18]:
for i in range(0,10):
    new_row = pd.DataFrame(data.loc[i,:].T)
    run = pd.concat([run,new_row], ignore_index=True)

In [19]:
new_row

,9
Unnamed: 0,6.0
Timestamp,17-09-2023 00:03
Pressure (psi),45.0
Temperature (°C),1000.0
Oil Flow (L/min),2.7
Vibration (mm/s),0.024
Power Status,ON
Oil Level (%),73.1
Tool RPM (rpm),1515
Machine Status,Running


In [20]:
run

,Unnamed: 0,Timestamp,Pressure (psi),Temperature (°C),Oil Flow (L/min),Vibration (mm/s),Power Status,Oil Level (%),Tool RPM (rpm),Machine Status,...,395,396,397,398,399,400,401,402,403,404
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
